In [1]:
import json
import pandas as pd
from tqdm import tqdm_notebook
import numpy as np
from tqdm import tqdm_notebook
from shapely.wkt import loads
from shapely.geometry import Point
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET

In [2]:
gdf = pd.read_csv('./data/gdf_s010_p010_v010.csv')
pbdata = pd.read_csv('./data/perth_base_2011.0.countscompare.tsv',sep='\t')
tree = ET.parse('./data/network.xml')
root = tree.getroot()

def parseAtts(atts):
    rd = {}
    for att in atts:
        name = att.attrib['name']
        rd[name] = att.text
    return rd




In [3]:
def parseLink(root):
    rd  = {'id':[], 'from':[], 'to':[], 'length':[],'freespeed':[],'capacity':[],'permlanes':[],'oneway':[],'modes':[],'origid':[],'type':[]}
    links = root[1]
    for link in links:
        attrdict = parseAtts(link[0])
        linkd = {**attrdict, **link.attrib}
        for key in linkd:
            rd[key].append(linkd[key])
    return rd

linkd = parseLink(root)
linkdf =  pd.DataFrame(linkd)
linkdf.to_csv('./data/linkdf.csv',index=False)

In [4]:
def getNodesFromTree(root):
    rd = {'id':[],'x':[],'y':[]}
    nodes = root[0]
    for node in nodes:
        rd['id'].append(node.attrib['id'])
        rd['x'].append(node.attrib['x'])
        rd['y'].append(node.attrib['y'])
    return rd

In [5]:
rd = getNodesFromTree(root)
nodedf = pd.DataFrame(rd)
nodedf.to_csv('./data/nodedf.csv',index=False)

In [6]:
nodedf = nodedf.set_index('id')
def getxyById(nodeid):
    return [nodedf.at[nodeid,'x'], nodedf.at[nodeid,'y']]
for idx,row in linkdf.iterrows():
    fromid = row['from']
    toid = row['to']
    fromxy = getxyById(fromid)
    toxy = getxyById(toid)
    linkdf.at[idx,'x0'] = fromxy[0]
    linkdf.at[idx,'y0'] = fromxy[1]
    linkdf.at[idx,'x1'] = toxy[0]
    linkdf.at[idx,'y1'] = toxy[1]
linkdf.to_csv('./data/linknode.csv',index=False)

In [7]:
mlink = pd.read_csv('./data/gdf_s010_p010_v010.csv')
link = pd.read_csv('./data/linknode.csv')

In [8]:
def parsePointX(pstr):
    xy = pstr.strip('(').strip(')').split(',')
    return float(xy[0])

def parsePointY(pstr):
    xy = pstr.strip('(').strip(')').split(',')
    return float(xy[1])
parsePointX('(115.821391, -31.9752)'), parsePointY('(115.821391, -31.9752)')

(115.821391, -31.9752)

In [9]:
def parsePolyline(polylinestr):
    return loads(polylinestr)

In [10]:
mlink['polyline'] = mlink['polyline'].map(parsePolyline)

In [11]:
link = link[(link['type'] == 'motorway') | (link['type'] == 'trunk')].reset_index()

In [12]:

for idx, row in tqdm_notebook(link.iterrows()):
    miniMlink = -1
    miniDistance = 999999
    for midx in mlink.index:
        pl = mlink.at[midx, 'polyline']
        mid = mlink.at[midx, 'mlink']
        dsum = Point(row['x0'], row['y0']).distance(pl) + Point(row['x1'], row['y1']).distance(pl)
        if dsum < miniDistance:
            miniMlink = mid
            miniDistance = dsum
    link.at[idx, 'pairedMlink'] = miniMlink
    link.at[idx,'distanceSum'] = miniDistance
    
link['pairedMlink'] = link.pairedMlink.astype(int)
link.to_csv('./data/mlinks_links_connected_total.csv',index=False)

In [13]:
countdf = pd.read_csv('./data/perth_base_2011.0.countscompare.tsv',sep='\t')
mldf = pd.read_csv('./data/mlinks_links_connected_total.csv')
mlinks = pd.read_csv('./data/gdf_s010_p010_v010.csv')

In [14]:
mldf = mldf[mldf['distanceSum'] < 0.001]
mldf.drop('index',axis = 1, inplace=True)
mlinks.drop(['IRIS_PolyLine','Start_Point','End_Point','NPI_Link_ID','SCATS_Link_Name','Notes'],axis = 1, inplace=True)

In [15]:
resdf = countdf.drop(['Count Station Id','Count volumes','Relative Error','Normalized Relative Error','GEH'],axis = 1)
resdf= resdf.reset_index()
for idx,row in mldf.iterrows():
    linkid = row['id']
    mlinkid = row['pairedMlink']
    resdf.loc[resdf['Link Id']==linkid,'mlink'] = mlinkid
res = resdf.dropna().reset_index()
res.drop(['level_0','index'],axis = 1, inplace=True)
res['mlink'] = res['mlink'].astype(int)
res.to_csv('./data/link-mlink-count-primary.csv',index=False)

In [60]:
resfull = res.merge(mlinks, on='mlink')
resfull.to_csv('./data/full-link-mlink-count-primary.csv',index=False)

AttributeError: 'dict' object has no attribute 'merge'

In [63]:
df = pd.read_csv('./data/full-link-mlink-count-primary.csv')
links = pd.read_csv('./data/linknode.csv')
links = links.set_index('id')
res = {}

df2 = df[['Link Id','Route_Name','MATSIM volumes']].groupby('Link Id').sum().reset_index()
dfm = pd.merge(df[['Link Id','Route_Name']],df2,on="Link Id")
dfms = dfm.sort_values(['Route_Name','MATSIM volumes'],ascending =False).reset_index()
df2 = df2.set_index('Link Id')
validLinkids={
    1:29993,
    2:22985,
    3:41703,
    4:50816,
    5:36618,
    9:40484,
    10:20351,
    11:53692,
    13:11227,
    15:32627,
    17:60243,
    18:35021,
    19:6581,
    20:44844,
    21:11353,
    22:15556,
    23:54908,
    24:12851,
    26:39983,
    27:1733,
}

for idx in tqdm_notebook(df.index):
    lid = df.at[idx, 'Link Id']
    if lid not in validLinkids.values():
        continue
    key = str(lid)
    if key not in res:
        desc = ['linkid:' + str(lid), 'mlink:' + str(df.at[idx, 'mlink']),'Route:'+ df.at[idx, 'Route_Name'],'Sum vol:' + str(df2.at[lid,'MATSIM volumes']), str(df.at[idx, 'Hour']) + ":" +str(df.at[idx, 'MATSIM volumes'])]
        desc = '<br/>'.join(desc)
        res[key] = {
            'desc':desc,
            'loc': [links.at[lid, 'y0'],links.at[lid, 'x0']]
        }
    else:
        res[key]['desc'] += "<br/>" + str(df.at[idx, 'Hour']) + ":" +str(df.at[idx, 'MATSIM volumes'])
        
jsonstr = json.dumps(res)
jsonstr = "var mydata = " + jsonstr + ";"
with open('./data/data_obs.js','w') as f:
    f.write(jsonstr)

In [64]:
highest_vol_linkids_foreachroute = []
rn = ""
for idx in dfms.index:
    crn = dfms.at[idx,'Route_Name']
    if crn != rn:
        highest_vol_linkids_foreachroute.append(dfms.at[idx,'Link Id'])
    rn = crn

In [65]:
df = pd.read_csv('./data/full-link-mlink-count-primary.csv')
links = pd.read_csv('./data/linknode.csv')
links = links.set_index('id')
res = {}

for idx in tqdm_notebook(df.index):
    lid = df.at[idx, 'Link Id']
    if lid not in highest_vol_linkids_foreachroute:
        continue
    key = str(lid)
    if key not in res:
        desc = ['linkid:' + str(lid), 'mlink:' + str(df.at[idx, 'mlink']),'Route:'+ df.at[idx, 'Route_Name'],'Sum vol:' + str(df2.at[lid,'MATSIM volumes']), str(df.at[idx, 'Hour']) + ":" +str(df.at[idx, 'MATSIM volumes'])]
        desc = '<br/>'.join(desc)
        res[key] = {
            'desc':desc,
            'loc': [links.at[lid, 'y0'],links.at[lid, 'x0']]
        }
    else:
        res[key]['desc'] += "<br/>" + str(df.at[idx, 'Hour']) + ":" +str(df.at[idx, 'MATSIM volumes'])
        
jsonstr = json.dumps(res)
jsonstr = "var mydata = " + jsonstr + ";"
with open('./data/data_maxvol.js','w') as f:
    f.write(jsonstr)